In [1]:
import csv
import pandas as pd
# Read the file Dataset_Concrete.csv and name it df.
df = pd.read_csv("C:\\Users\\Kimble\\Downloads\\dynamo_games_initial.csv")
print(df)

     Year                                                URL  \
0    2019  https://www.houstondynamofc.com/competitions/m...   
1    2019  https://www.houstondynamofc.com/competitions/c...   
2    2019  https://www.houstondynamofc.com/competitions/m...   
3    2019  https://www.houstondynamofc.com/competitions/m...   
4    2019  https://www.houstondynamofc.com/competitions/m...   
..    ...                                                ...   
107  2024  https://www.houstondynamofc.com/competitions/m...   
108  2024  https://www.houstondynamofc.com/competitions/m...   
109  2024  https://www.houstondynamofc.com/competitions/m...   
110  2024  https://www.houstondynamofc.com/competitions/m...   
111  2024  https://www.houstondynamofc.com/competitions/m...   

                            Date             Competition     Opponent  \
0          Saturday March 2 2019      MLS Regular Season    Salt Lake   
1           Tuesday March 5 2019  CONCACAF Champions Cup       Tigres   
2          S

In [2]:
import pandas as pd
from datetime import datetime
import requests

# Sample dataframe df with a 'date' column
# df = pd.DataFrame({'date': ['Saturday March 2 2019', 'Sunday March 3 2019']})

# Assuming df is your existing dataframe
# Ensure that df has a 'date' column in the format "Saturday March 2 2019"

# Parse the 'date' column to standard date format
def parse_date(date_str):
    try:
        date_obj = datetime.strptime(date_str, '%A %B %d %Y')
        return date_obj
    except ValueError:
        # Handle error if date format does not match
        print(f"Date format error with date: {date_str}")
        return None

df['date_parsed'] = df['Date'].apply(parse_date)
df['date_str'] = df['date_parsed'].dt.strftime('%Y-%m-%d')

# Drop rows where date_parsed is None
df = df[df['date_parsed'].notnull()]

# Get unique dates
unique_dates = df['date_str'].unique()

weather_data = []
api_key = 'Z6N2HFYZV72Y6NP7BNTTSYPMZ'  # Replace with your actual API key

for date in unique_dates:
    # Build the API URL
    url = f'https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/Houston/{date}?unitGroup=metric&key={api_key}&include=days'

    # Make the request
    response = requests.get(url)

    # Check if the request was successful
    if response.status_code == 200:
        data = response.json()
        # Extract the temperature and rain data
        if 'days' in data and len(data['days']) > 0:
            day_data = data['days'][0]  # assuming days[0] corresponds to the date
            temp = day_data.get('temp', None)
            # For rain, check if 'preciptype' includes 'rain'
            preciptype = day_data.get('preciptype') or []
            rain = 1 if 'rain' in preciptype else 0

            # Append the data to the list
            weather_data.append({'date_str': date, 'temperature': temp, 'rain': rain})
        else:
            print(f"No day data available for date {date}")
            weather_data.append({'date_str': date, 'temperature': None, 'rain': None})
    else:
        print(f"Failed to retrieve data for date {date}")
        weather_data.append({'date_str': date, 'temperature': None, 'rain': None})

# Create a dataframe from weather_data
weather_df = pd.DataFrame(weather_data)

# Merge the weather_df with the original df on 'date_str'
df_merged = pd.merge(df, weather_df, on='date_str', how='left')

# Drop temporary columns if not needed
df_merged = df_merged.drop(columns=['date_parsed', 'date_str'])

# Display the merged dataframe
print(df_merged)

     Year                                                URL  \
0    2019  https://www.houstondynamofc.com/competitions/m...   
1    2019  https://www.houstondynamofc.com/competitions/c...   
2    2019  https://www.houstondynamofc.com/competitions/m...   
3    2019  https://www.houstondynamofc.com/competitions/m...   
4    2019  https://www.houstondynamofc.com/competitions/m...   
..    ...                                                ...   
107  2024  https://www.houstondynamofc.com/competitions/m...   
108  2024  https://www.houstondynamofc.com/competitions/m...   
109  2024  https://www.houstondynamofc.com/competitions/m...   
110  2024  https://www.houstondynamofc.com/competitions/m...   
111  2024  https://www.houstondynamofc.com/competitions/m...   

                            Date             Competition     Opponent  \
0          Saturday March 2 2019      MLS Regular Season    Salt Lake   
1           Tuesday March 5 2019  CONCACAF Champions Cup       Tigres   
2          S

In [3]:
len(df_merged)

112

In [4]:
import os

# Path to your original CSV file
original_file_path = "C:\\Users\\Kimble\\Downloads\\dynamo_games_initial.csv"

# Extract the directory from the original file path
directory = os.path.dirname(original_file_path)

# Define the new file path in the same directory
new_file_path = os.path.join(directory, 'New_dynamo_data.csv')

# Write the df_merged dataframe to the new CSV file
df_merged.to_csv(new_file_path, index=False)